In [1]:
import os
import time
import torch
import torch as tc
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import numpy as np

In [2]:
import models
from models import weights_init

# Main code

In [3]:
#Libera as funcionalidades da biblioteca cudnn
cudnn.benchmark = True

use_gpu = torch.cuda.is_available()
#use_gpu = False
if use_gpu:
    print("You are using CUDA. If it is not what you want, manually set this as False!")

You are using CUDA. If it is not what you want, manually set this as False!


In [4]:
nc = 3
ngpu = 1
nz = 100
ngf = 64
ndf = 64
n_extra_d = 0
n_extra_g = 1 # Aqui a jogada é que o gerador deve ser mais poderoso q o detetive
imageSize = 64

In [5]:
!cd images && pwd


/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/images


## Setando as transformações

In [6]:
!ls images/images2/

ls: cannot access 'images/images2/': No such file or directory


In [7]:
!ls dataset

anime-faces


In [8]:
dataset = dset.ImageFolder(
    root='/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/dataset/anime-faces',
    transform=transforms.Compose([
            transforms.Scale((imageSize, imageSize)),
            # transforms.CenterCrop(opt.imageSize),
            transforms.ToTensor(),
            #transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)), # bring images to (-1,1)
        ])
)

## Setando o Dataloader

In [9]:
batch_size=64
num_workers = 1

In [33]:
dataloader = tc.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)
#126 classes

## Setando o modelo

### Modelo DCGAN

In [11]:
class _netD_1(nn.Module):
    def __init__(self, ngpu, nz, nc, ndf,  n_extra_layers_d):
        super(_netD_1, self).__init__()
        self.ngpu = ngpu
        main = nn.Sequential(
            # input is (nc) x 96 x 96
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False), # 5,3,1 for 96x96
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
        )

        # Extra layers
        for t in range(n_extra_layers_d):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, ndf * 8),
                            nn.Conv2d(ndf * 8, ndf * 8, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, ndf * 8),
                            nn.BatchNorm2d(ndf * 8))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, ndf * 8),
                            nn.LeakyReLU(0.2, inplace=True))


        main.add_module('final_layers.conv', nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False))
        main.add_module('final_layers.sigmoid', nn.Sigmoid())
        # state size. 1 x 1 x 1
        self.main = main

    def forward(self, input):
        gpu_ids = None
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            gpu_ids = range(self.ngpu)
        output = nn.parallel.data_parallel(self.main, input, gpu_ids)
        return output.view(-1, 1)


In [12]:
# DCGAN model, fully convolutional architecture
class _netG_1(nn.Module):
    def __init__(self, ngpu, nz, nc , ngf, n_extra_layers_g):
        super(_netG_1, self).__init__()
        self.ngpu = ngpu
        #self.nz = nz
        #self.nc = nc
        #self.ngf = ngf
        main = nn.Sequential(
            # input is Z, going into a convolution
            # state size. nz x 1 x 1
            nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ngf) x 32 x 32
        )

        # Extra layers
        for t in range(n_extra_layers_g):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, ngf),
                            nn.Conv2d(ngf, ngf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, ngf),
                            nn.BatchNorm2d(ngf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, ngf),
                            nn.LeakyReLU(0.2, inplace=True))

        main.add_module('final_layer.deconv', 
        	             nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False)) # 5,3,1 for 96x96
        main.add_module('final_layer.tanh', 
        	             nn.Tanh())
            # state size. (nc) x 96 x 96

        self.main = main


    def forward(self, input):
        gpu_ids = None
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            gpu_ids = range(self.ngpu)
        return nn.parallel.data_parallel(self.main, input, gpu_ids), 0

In [13]:
netG = _netG_1(ngpu, nz, nc, ngf, n_extra_g)


In [14]:
print(netG)

_netG_1 (
  (main): Sequential (
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): LeakyReLU (0.2, inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): LeakyReLU (0.2, inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): LeakyReLU (0.2, inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): LeakyReLU (0.2, inplace)
    (extra-layers-0.64.conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (extra-layers-0.64.batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  

In [15]:
netD = _netD_1(ngpu, nz, nc, ndf, n_extra_d)

In [16]:
print(netD)

_netD_1 (
  (main): Sequential (
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU (0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU (0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU (0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU (0.2, inplace)
    (final_layers.conv): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (final_layers.sigmoid): Sigmoid ()
  )
)


# Carregando pesos pré-treinados

In [17]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


In [18]:
#Parece ser um inicializador de pesos hardcoded
netG.apply(weights_init)
netD.apply(weights_init)
print(())

()


In [19]:
load=False
if load:
    netD.load_state_dict(tc.load('path_d'))
    netG.load_state_dict(torch.load('path_G'))


### Parametros de treinamento

In [20]:
criterion = nn.BCELoss()
criterion_MSE = nn.MSELoss()


In [21]:

input = torch.FloatTensor(batch_size, 3, imageSize, imageSize)
print(input.size())
noise = torch.FloatTensor(batch_size, nz, 1, 1)
print(noise.size())

torch.Size([64, 3, 64, 64])
torch.Size([64, 100, 1, 1])


In [22]:
#parser.add_argument('--binary', action='store_true', help='z from bernoulli distribution, with prob=0.5')
binary=False
#Ele testa pergunta se vc quer que o seu Z venha da distribuição bernoulli
if binary:
    bernoulli_prob = torch.FloatTensor(batch_size, nz, 1, 1).fill_(0.5)
    fixed_noise = torch.bernoulli(bernoulli_prob)
else:
    fixed_noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)


In [23]:
label = torch.FloatTensor(batch_size)
real_label = 1
fake_label = 0


### Broadcast para CUDA, se quiser

In [24]:
if use_gpu:
    netD.cuda()
    netG.cuda()
    criterion = criterion.cuda()
    criterion_MSE = criterion_MSE.cuda()
    input,label = input.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

### Transformando tudo em variable

In [25]:

input = Variable(input)
label = Variable(label)
noise = Variable(noise)
fixed_noise = Variable(fixed_noise)


### Setando o optimizer

In [26]:
beta1, beta2 = 0.9,0.999
lr = 2.0e-4
optimizerD = optim.Adam(netD.parameters(), lr = lr, betas = (beta1, beta2))
optimizerG = optim.Adam(netG.parameters(), lr = lr, betas = (beta1, beta2))
outputDir = 'outputdir'

## Treinando !

In [27]:
print(type(input))
print(input.size())
print(label.size())

<class 'torch.autograd.variable.Variable'>
torch.Size([64, 3, 64, 64])
torch.Size([64])


In [30]:
def train_both_networks(num_epochs, dataloader, netD, netG, d_labelSmooth, outputDir, model_option =1,binary = False):
    use_gpu = tc.cuda.is_available()

    for epoch in range(num_epochs):
        for i, data in enumerate(dataloader, 0):
            start_iter = time.time()
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            # train with real
            netD.zero_grad()
            real_cpu, _ = data
            batch_size = real_cpu.size(0)
            input.data.resize_(real_cpu.size()).copy_(real_cpu)
            label.data.resize_(batch_size).fill_(real_label - d_labelSmooth) # use smooth label for discriminator

            output = netD(input)
            errD_real = criterion(output, label)
            errD_real.backward()
            D_x = output.data.mean()
            # train with fake
            noise.data.resize_(batch_size, nz, 1, 1)
            if binary:
                bernoulli_prob.resize_(noise.data.size())
                noise.data.copy_(2*(torch.bernoulli(bernoulli_prob)-0.5))
            else:
                noise.data.normal_(0, 1)
            fake,z_prediction = netG(noise)
            label.data.fill_(fake_label)
            output = netD(fake.detach()) # add ".detach()" to avoid backprop through G
            errD_fake = criterion(output, label)
            errD_fake.backward() # gradients for fake/real will be accumulated
            D_G_z1 = output.data.mean()
            errD = errD_real + errD_fake
            optimizerD.step() # .step() can be called once the gradients are computed

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            netG.zero_grad()
            label.data.fill_(real_label) # fake labels are real for generator cost
            output = netD(fake)
            errG = criterion(output, label)
            errG.backward(retain_variables=True) # True if backward through the graph for the second time
            if model_option == 2: # with z predictor
                errG_z = criterion_MSE(z_prediction, noise)
                errG_z.backward()
            D_G_z2 = output.data.mean()
            optimizerG.step()

            end_iter = time.time()
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f Elapsed %.2f s'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2, end_iter-start_iter))
            if i % 100 == 0:
                # the first 64 samples from the mini-batch are saved.
                vutils.save_image(real_cpu[0:64,:,:,:],
                        '%s/real_samples.png' % outputDir, nrow=8)
                fake,_ = netG(fixed_noise)
                vutils.save_image(fake.data[0:64,:,:,:],
                        '%s/fake_samples_epoch_%03d.png' % (outputDir, epoch), nrow=8)
        if epoch % 1 == 0:
            # do checkpointing
            torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outputDir, epoch))
            torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outputDir, epoch))

In [ ]:
num_epochs = 100
d_labelSmooth = 0.2

train_both_networks(num_epochs, dataloader, netD,netG,d_labelSmooth, outputDir)

/home/gabriel/anaconda3/envs/py35/lib/python3.5/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/gabriel/anaconda3/envs/py35/lib/python3.5/site-packages/torch/autograd/__init__.py:92: UserWarning: retain_variables option is deprecated and will be removed in 0.3. Use retain_graph instead.
  warnings.warn("retain_variables option is deprecated and will be removed in 0.3. "


[0/100][0/1799] Loss_D: 0.6416 Loss_G: 3.3060 D(x): 0.8684 D(G(z)): 0.0862 / 0.0456 Elapsed 0.06 s
[0/100][1/1799] Loss_D: 0.6161 Loss_G: 3.8739 D(x): 0.7729 D(G(z)): 0.0686 / 0.0319 Elapsed 0.06 s
[0/100][2/1799] Loss_D: 0.5861 Loss_G: 4.5308 D(x): 0.7384 D(G(z)): 0.0296 / 0.0157 Elapsed 0.06 s
[0/100][3/1799] Loss_D: 0.6078 Loss_G: 4.2900 D(x): 0.7054 D(G(z)): 0.0332 / 0.0229 Elapsed 0.06 s
[0/100][4/1799] Loss_D: 0.5875 Loss_G: 4.5135 D(x): 0.7283 D(G(z)): 0.0150 / 0.0138 Elapsed 0.06 s
[0/100][5/1799] Loss_D: 0.5823 Loss_G: 3.9553 D(x): 0.7423 D(G(z)): 0.0233 / 0.0250 Elapsed 0.06 s
[0/100][6/1799] Loss_D: 0.6302 Loss_G: 3.1136 D(x): 0.7246 D(G(z)): 0.0622 / 0.0652 Elapsed 0.06 s
[0/100][7/1799] Loss_D: 0.6455 Loss_G: 3.4272 D(x): 0.8341 D(G(z)): 0.0781 / 0.0489 Elapsed 0.06 s
[0/100][8/1799] Loss_D: 0.5858 Loss_G: 4.2946 D(x): 0.8274 D(G(z)): 0.0452 / 0.0201 Elapsed 0.06 s
[0/100][9/1799] Loss_D: 0.5865 Loss_G: 4.3363 D(x): 0.7670 D(G(z)): 0.0405 / 0.0182 Elapsed 0.06 s
[0/100][10

[0/100][85/1799] Loss_D: 0.7511 Loss_G: 2.5527 D(x): 0.8764 D(G(z)): 0.1616 / 0.0988 Elapsed 0.05 s
[0/100][86/1799] Loss_D: 0.7651 Loss_G: 3.5438 D(x): 0.8546 D(G(z)): 0.1562 / 0.0407 Elapsed 0.06 s
[0/100][87/1799] Loss_D: 0.6164 Loss_G: 4.9328 D(x): 0.7871 D(G(z)): 0.0446 / 0.0098 Elapsed 0.06 s
[0/100][88/1799] Loss_D: 0.6939 Loss_G: 5.5378 D(x): 0.5773 D(G(z)): 0.0089 / 0.0063 Elapsed 0.06 s
[0/100][89/1799] Loss_D: 0.8936 Loss_G: 3.4825 D(x): 0.4489 D(G(z)): 0.0149 / 0.0458 Elapsed 0.06 s
[0/100][90/1799] Loss_D: 0.6417 Loss_G: 2.4213 D(x): 0.8415 D(G(z)): 0.0814 / 0.1161 Elapsed 0.06 s
[0/100][91/1799] Loss_D: 0.8033 Loss_G: 3.0316 D(x): 0.9251 D(G(z)): 0.1315 / 0.0666 Elapsed 0.06 s
[0/100][92/1799] Loss_D: 0.6544 Loss_G: 3.8198 D(x): 0.8270 D(G(z)): 0.0903 / 0.0288 Elapsed 0.06 s
[0/100][93/1799] Loss_D: 0.6263 Loss_G: 4.6539 D(x): 0.7765 D(G(z)): 0.0491 / 0.0143 Elapsed 0.06 s
[0/100][94/1799] Loss_D: 0.5621 Loss_G: 5.4796 D(x): 0.7973 D(G(z)): 0.0209 / 0.0066 Elapsed 0.06 s


[0/100][167/1799] Loss_D: 0.7126 Loss_G: 3.1946 D(x): 0.9050 D(G(z)): 0.1159 / 0.0544 Elapsed 0.06 s
[0/100][168/1799] Loss_D: 0.6418 Loss_G: 4.3357 D(x): 0.8683 D(G(z)): 0.0713 / 0.0189 Elapsed 0.06 s
[0/100][169/1799] Loss_D: 0.5573 Loss_G: 5.4926 D(x): 0.7787 D(G(z)): 0.0202 / 0.0054 Elapsed 0.06 s
[0/100][170/1799] Loss_D: 0.7068 Loss_G: 4.5834 D(x): 0.5557 D(G(z)): 0.0143 / 0.0134 Elapsed 0.06 s
[0/100][171/1799] Loss_D: 0.6411 Loss_G: 4.2711 D(x): 0.6456 D(G(z)): 0.0153 / 0.0249 Elapsed 0.06 s
[0/100][172/1799] Loss_D: 0.5855 Loss_G: 3.4417 D(x): 0.7743 D(G(z)): 0.0334 / 0.0464 Elapsed 0.06 s
[0/100][173/1799] Loss_D: 0.6785 Loss_G: 3.1274 D(x): 0.8823 D(G(z)): 0.0967 / 0.0560 Elapsed 0.06 s
[0/100][174/1799] Loss_D: 0.5873 Loss_G: 4.6931 D(x): 0.8330 D(G(z)): 0.0318 / 0.0129 Elapsed 0.06 s
[0/100][175/1799] Loss_D: 0.6467 Loss_G: 4.0695 D(x): 0.6950 D(G(z)): 0.0462 / 0.0254 Elapsed 0.06 s
[0/100][176/1799] Loss_D: 0.5727 Loss_G: 5.0738 D(x): 0.7865 D(G(z)): 0.0184 / 0.0100 Elaps

[0/100][249/1799] Loss_D: 0.6005 Loss_G: 4.0530 D(x): 0.7269 D(G(z)): 0.0411 / 0.0246 Elapsed 0.06 s
[0/100][250/1799] Loss_D: 0.5880 Loss_G: 4.5305 D(x): 0.7621 D(G(z)): 0.0315 / 0.0189 Elapsed 0.06 s
[0/100][251/1799] Loss_D: 0.6317 Loss_G: 4.5223 D(x): 0.6504 D(G(z)): 0.0145 / 0.0147 Elapsed 0.06 s
[0/100][252/1799] Loss_D: 0.5675 Loss_G: 4.1067 D(x): 0.7560 D(G(z)): 0.0195 / 0.0211 Elapsed 0.06 s
[0/100][253/1799] Loss_D: 0.5839 Loss_G: 4.0746 D(x): 0.8382 D(G(z)): 0.0276 / 0.0229 Elapsed 0.06 s
[0/100][254/1799] Loss_D: 0.6235 Loss_G: 4.0023 D(x): 0.8636 D(G(z)): 0.0512 / 0.0268 Elapsed 0.06 s
[0/100][255/1799] Loss_D: 0.6770 Loss_G: 3.7254 D(x): 0.6231 D(G(z)): 0.0340 / 0.0323 Elapsed 0.06 s
[0/100][256/1799] Loss_D: 0.6369 Loss_G: 3.4169 D(x): 0.6737 D(G(z)): 0.0310 / 0.0450 Elapsed 0.06 s
[0/100][257/1799] Loss_D: 0.6452 Loss_G: 3.2601 D(x): 0.8827 D(G(z)): 0.0678 / 0.0511 Elapsed 0.06 s
[0/100][258/1799] Loss_D: 0.6229 Loss_G: 3.9828 D(x): 0.8665 D(G(z)): 0.0602 / 0.0260 Elaps

...

[14/100][582/1799] Loss_D: 0.5852 Loss_G: 4.3071 D(x): 0.8746 D(G(z)): 0.0306 / 0.0167 Elapsed 0.06 s
[14/100][583/1799] Loss_D: 0.5767 Loss_G: 4.1383 D(x): 0.7041 D(G(z)): 0.0277 / 0.0214 Elapsed 0.06 s
[14/100][584/1799] Loss_D: 0.5531 Loss_G: 4.5856 D(x): 0.8305 D(G(z)): 0.0205 / 0.0125 Elapsed 0.06 s
[14/100][585/1799] Loss_D: 0.5598 Loss_G: 4.4125 D(x): 0.7250 D(G(z)): 0.0174 / 0.0148 Elapsed 0.06 s
[14/100][586/1799] Loss_D: 0.5583 Loss_G: 4.3105 D(x): 0.7347 D(G(z)): 0.0152 / 0.0160 Elapsed 0.06 s
[14/100][587/1799] Loss_D: 0.5553 Loss_G: 4.4692 D(x): 0.8407 D(G(z)): 0.0172 / 0.0136 Elapsed 0.06 s
[14/100][588/1799] Loss_D: 0.5384 Loss_G: 4.8515 D(x): 0.8197 D(G(z)): 0.0147 / 0.0098 Elapsed 0.06 s
[14/100][589/1799] Loss_D: 0.5773 Loss_G: 5.1921 D(x): 0.8846 D(G(z)): 0.0155 / 0.0074 Elapsed 0.06 s
[14/100][590/1799] Loss_D: 0.6708 Loss_G: 4.3941 D(x): 0.5759 D(G(z)): 0.0127 / 0.0153 Elapsed 0.06 s
[14/100][591/1799] Loss_D: 0.5565 Loss_G: 4.4584 D(x): 0.8514 D(G(z)): 0.0164 / 0.

[14/100][664/1799] Loss_D: 0.6627 Loss_G: 3.9529 D(x): 0.9213 D(G(z)): 0.0566 / 0.0240 Elapsed 0.06 s
[14/100][665/1799] Loss_D: 0.5456 Loss_G: 5.0124 D(x): 0.8077 D(G(z)): 0.0213 / 0.0083 Elapsed 0.06 s
[14/100][666/1799] Loss_D: 0.5572 Loss_G: 5.2371 D(x): 0.7201 D(G(z)): 0.0129 / 0.0069 Elapsed 0.06 s
[14/100][667/1799] Loss_D: 0.6738 Loss_G: 5.1042 D(x): 0.5668 D(G(z)): 0.0056 / 0.0079 Elapsed 0.06 s
[14/100][668/1799] Loss_D: 0.5781 Loss_G: 3.6699 D(x): 0.6744 D(G(z)): 0.0180 / 0.0328 Elapsed 0.06 s
[14/100][669/1799] Loss_D: 0.5741 Loss_G: 3.7089 D(x): 0.8796 D(G(z)): 0.0280 / 0.0319 Elapsed 0.06 s
[14/100][670/1799] Loss_D: 0.6473 Loss_G: 3.4720 D(x): 0.9116 D(G(z)): 0.0592 / 0.0371 Elapsed 0.06 s
[14/100][671/1799] Loss_D: 0.5591 Loss_G: 4.1461 D(x): 0.8059 D(G(z)): 0.0365 / 0.0202 Elapsed 0.06 s
[14/100][672/1799] Loss_D: 0.5398 Loss_G: 4.8819 D(x): 0.7532 D(G(z)): 0.0155 / 0.0096 Elapsed 0.06 s
[14/100][673/1799] Loss_D: 0.5439 Loss_G: 5.1073 D(x): 0.7952 D(G(z)): 0.0124 / 0.

[14/100][746/1799] Loss_D: 0.6585 Loss_G: 2.9647 D(x): 0.8974 D(G(z)): 0.0885 / 0.0602 Elapsed 0.06 s
[14/100][747/1799] Loss_D: 0.7460 Loss_G: 3.7290 D(x): 0.9391 D(G(z)): 0.0995 / 0.0293 Elapsed 0.06 s
[14/100][748/1799] Loss_D: 0.5680 Loss_G: 5.2301 D(x): 0.8542 D(G(z)): 0.0254 / 0.0068 Elapsed 0.06 s
[14/100][749/1799] Loss_D: 0.5737 Loss_G: 5.4970 D(x): 0.6946 D(G(z)): 0.0097 / 0.0052 Elapsed 0.06 s
[14/100][750/1799] Loss_D: 0.8048 Loss_G: 4.2235 D(x): 0.4533 D(G(z)): 0.0086 / 0.0187 Elapsed 0.06 s
[14/100][751/1799] Loss_D: 0.5844 Loss_G: 4.0160 D(x): 0.8892 D(G(z)): 0.0219 / 0.0239 Elapsed 0.06 s
[14/100][752/1799] Loss_D: 0.5827 Loss_G: 4.1335 D(x): 0.8810 D(G(z)): 0.0251 / 0.0199 Elapsed 0.06 s
[14/100][753/1799] Loss_D: 0.5502 Loss_G: 4.6497 D(x): 0.8371 D(G(z)): 0.0209 / 0.0123 Elapsed 0.06 s
[14/100][754/1799] Loss_D: 0.5442 Loss_G: 5.1124 D(x): 0.7858 D(G(z)): 0.0122 / 0.0082 Elapsed 0.06 s
[14/100][755/1799] Loss_D: 0.5714 Loss_G: 4.7093 D(x): 0.7104 D(G(z)): 0.0116 / 0.

[14/100][828/1799] Loss_D: 0.5698 Loss_G: 3.5224 D(x): 0.8271 D(G(z)): 0.0471 / 0.0351 Elapsed 0.06 s
[14/100][829/1799] Loss_D: 0.5780 Loss_G: 4.1749 D(x): 0.8359 D(G(z)): 0.0397 / 0.0220 Elapsed 0.06 s
[14/100][830/1799] Loss_D: 0.5837 Loss_G: 4.5098 D(x): 0.7018 D(G(z)): 0.0174 / 0.0142 Elapsed 0.06 s
[14/100][831/1799] Loss_D: 0.6109 Loss_G: 3.7808 D(x): 0.6435 D(G(z)): 0.0207 / 0.0283 Elapsed 0.06 s
[14/100][832/1799] Loss_D: 0.5827 Loss_G: 3.5085 D(x): 0.8540 D(G(z)): 0.0419 / 0.0372 Elapsed 0.06 s
[14/100][833/1799] Loss_D: 0.5714 Loss_G: 4.1174 D(x): 0.8572 D(G(z)): 0.0338 / 0.0210 Elapsed 0.06 s
[14/100][834/1799] Loss_D: 0.5342 Loss_G: 4.8211 D(x): 0.7926 D(G(z)): 0.0161 / 0.0101 Elapsed 0.06 s
[14/100][835/1799] Loss_D: 0.5411 Loss_G: 4.8174 D(x): 0.8063 D(G(z)): 0.0184 / 0.0104 Elapsed 0.06 s
[14/100][836/1799] Loss_D: 0.5554 Loss_G: 4.9961 D(x): 0.7264 D(G(z)): 0.0119 / 0.0091 Elapsed 0.06 s
[14/100][837/1799] Loss_D: 0.5961 Loss_G: 4.5326 D(x): 0.6561 D(G(z)): 0.0114 / 0.

[14/100][910/1799] Loss_D: 0.7008 Loss_G: 3.8542 D(x): 0.9190 D(G(z)): 0.0921 / 0.0270 Elapsed 0.06 s
[14/100][911/1799] Loss_D: 0.5622 Loss_G: 5.3114 D(x): 0.7325 D(G(z)): 0.0187 / 0.0064 Elapsed 0.06 s
[14/100][912/1799] Loss_D: 0.6424 Loss_G: 4.0035 D(x): 0.6053 D(G(z)): 0.0255 / 0.0225 Elapsed 0.06 s
[14/100][913/1799] Loss_D: 0.5763 Loss_G: 4.0290 D(x): 0.6800 D(G(z)): 0.0175 / 0.0233 Elapsed 0.06 s
[14/100][914/1799] Loss_D: 0.5487 Loss_G: 3.8243 D(x): 0.8077 D(G(z)): 0.0234 / 0.0279 Elapsed 0.06 s
[14/100][915/1799] Loss_D: 0.5882 Loss_G: 3.8586 D(x): 0.8672 D(G(z)): 0.0347 / 0.0263 Elapsed 0.06 s
[14/100][916/1799] Loss_D: 0.5922 Loss_G: 3.6538 D(x): 0.7997 D(G(z)): 0.0573 / 0.0327 Elapsed 0.06 s
[14/100][917/1799] Loss_D: 0.5574 Loss_G: 4.9343 D(x): 0.7982 D(G(z)): 0.0186 / 0.0094 Elapsed 0.06 s
[14/100][918/1799] Loss_D: 0.5441 Loss_G: 5.3918 D(x): 0.7632 D(G(z)): 0.0106 / 0.0062 Elapsed 0.06 s
[14/100][919/1799] Loss_D: 0.5542 Loss_G: 5.3051 D(x): 0.7412 D(G(z)): 0.0083 / 0.